In [136]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import math
import os

maxAcc = 0.0
maxIter = 0
C_Lambda = 0.03
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10
M = 10
PHI = []
IsSynthetic = False

# Create Pair Data by Concatenating Features

In [137]:
# This method takes in the image pair dataset(df_pair) along with the feature dataset(df). 
# It then joins df with df_pair on image_id_A and image_id_B thus creating a dataframe 
# with allthe features of image A and image B i.e. 18 features.
def create_pair_data_concat(df, df_pair):
    df.rename(columns={'img_id': 'img_id_A'}, inplace=True)
    df_new = pd.merge(df_pair, df, on="img_id_A")
    df.rename(columns={'img_id_A': 'img_id_B'}, inplace=True)
    df_new = pd.merge(df_new, df, on="img_id_B")
    df.rename(columns={'img_id_B': 'img_id'}, inplace=True)
    return df_new  

# Create Pair Data by Subtracting Features

In [138]:
# This method takes in the image pair dataset(df_final) along with the feature dataset(df). 
# It then joins df with df_final on image_id_A and image_id_B separately and performs 
# feature subtraction forming a dataframe consisting of 9 features.
def create_pair_data_subtract(df, df_final):
    df_target = df_final['target']
    df.rename(columns={'img_id': 'img_id_A'}, inplace=True)
    df1 = pd.merge(df_final, df, on="img_id_A")
    df1.drop(['img_id_A','img_id_B','target' ], axis=1, inplace=True)
    df.rename(columns={'img_id_A': 'img_id_B'}, inplace=True)
    df2 = pd.merge(df_final, df, on="img_id_B")
    df.rename(columns={'img_id_B': 'img_id'}, inplace=True)
    df2.drop(['img_id_A','img_id_B','target' ], axis=1, inplace=True)
    df_sub = df1.sub(df2)
    df_final = np.absolute(df_sub)
    return df_final, df_target

# Generate Training, Validation data and target

In [139]:
# This method accepts the entire training target data and splits it into the valPercent passed into it.
def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    return t

# This method accepts the entire training data and splits it into the valPercent passed into it.
def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    return d2

# This method accepts the entire validation data and splits it into the valPercent passed into it.
def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    return dataMatrix

# This method accepts the entire validation target data and splits it into the valPercent passed into it.
def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    return t

# Generate Phi Matrix

In [140]:
def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic):
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))        
    varVect     = []
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:
        BigSigma = np.dot(3,BigSigma)
    else:
        BigSigma = np.dot(200,BigSigma)
    print ("BigSigma Generated..")
    return BigSigma

def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow)
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L

def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    DataT = np.transpose(Data) 
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01)) 
    PHI = np.zeros((int(TrainingLen),len(MuMatrix)))
    BigSigInv = np.linalg.pinv(BigSigma) 
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    return PHI

def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    return Y

def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

# Generate Raw Data

In [141]:
# This method takes in the feature data(df), the same pairs and different pairs to create the 
# feature concat and subtract dataset. For both operations, we select the same number of same 
# and different pair datasets and append the rows and randomize the dataset. We merge the 
# dataframes to concat/subtract the features and remove the columns where the feature 
# values accross the dataset iszero or variance is zero.
def generate_raw_data(df, df_diff, df_same, feature_operation, dataType):
    if feature_operation == "Concat":
        df_diff = df_diff.sample(n=df_same.shape[0])
        df_final = pd.concat([df_same, df_diff]).sample(frac=1).reset_index(drop=True)
        df_final = create_pair_data_concat(df, df_final)
        df_target = df_final['target']
        df_final.drop(['img_id_A','img_id_B','target' ], axis=1, inplace=True)
    if feature_operation == "Subtract":
        df_diff = df_diff.sample(n=df_same.shape[0])
        df_final = pd.concat([df_same, df_diff]).sample(frac=1).reset_index(drop=True)
        df_final, df_target = create_pair_data_subtract(df, df_final)
    uniques = df_final.apply(lambda x: x.nunique())
    df_final = df_final.drop(uniques[uniques==1].index, axis=1)
    return df_final, df_target   

# Perform Stochastic Gradient Descent

In [142]:
# ------------- Implementation for Stochastic Gradient Descent --------------- #
# we randomize the weights by multiplying a scalar value with the weights obtained from the closed form solution.
def generate_ERMS(TrainingTarget, TRAINING_PHI, VAL_PHI, TEST_PHI, ValDataAct, TestDataAct, dataType):
    if dataType == "GSC":
        range_loop = 200
    else:
        range_loop = TrainingTarget.shape[0]
    W_Now = np.ones(M)
    La           = 2
    learningRate = 0.01
    L_Erms_Val   = []
    L_Erms_TR    = []
    L_Erms_Test  = []
    W_Mat        = []
    print(TRAINING_PHI[0].shape)
    for i in range(0,range_loop):
        
        Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
        La_Delta_E_W  = np.dot(La,W_Now)
        Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
        Delta_W       = -np.dot(learningRate,Delta_E)
        W_T_Next      = W_Now + Delta_W
        W_Now         = W_T_Next
        
        #-----------------TrainingData Accuracy---------------------#
        TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
        Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
        L_Erms_TR.append(float(Erms_TR.split(',')[1]))

        #-----------------ValidationData Accuracy---------------------#
        VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
        Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
        L_Erms_Val.append(float(Erms_Val.split(',')[1]))

        #-----------------TestingData Accuracy---------------------#
        TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
        Erms_Test = GetErms(TEST_OUT,TestDataAct)
        L_Erms_Test.append(float(Erms_Test.split(',')[1]))
        
    print ('----------Gradient Descent Solution--------------------')
    print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
    print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
    print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))     

# Clustering and Training Model

In [143]:
def train_model(RawData, RawTarget, dataType):
    # Create the training dataset and target data with 80% of the raw dataset
    TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
    TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
    
    # Create the validation dataset and target data with 10% of the raw dataset
    ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
    ValData    = GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))
    
    # Create the testing dataset and target data with 10% of the raw dataset
    TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
    TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
    
    ErmsArr = []
    AccuracyArr = []
    kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData))
    Mu = kmeans.cluster_centers_
    BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)
    TRAINING_PHI = GetPhiMatrix(RawData, Mu, BigSigma, TrainingPercent)
    TEST_PHI     = GetPhiMatrix(TestData, Mu, BigSigma, 100) 
    VAL_PHI      = GetPhiMatrix(ValData, Mu, BigSigma, 100)
    
    generate_ERMS(TrainingTarget, TRAINING_PHI, VAL_PHI, TEST_PHI, ValDataAct, TestDataAct, dataType)

In [144]:
df_hof = pd.read_csv("HOF/HumanObserved-Features-Data.csv",index_col=0)
df_diff_hof = pd.read_csv("HOF/diffn_pairs.csv")
df_same_hof = pd.read_csv("HOF/same_pairs.csv")

print("-----------------------HOF : Concat--------------------")
df_final, df_target = generate_raw_data(df_hof, df_diff_hof, df_same_hof, "Concat", "HOF")
RawData = np.transpose(df_final.as_matrix())
RawTarget = df_target.as_matrix()
train_model(RawData, RawTarget, "HOF")

print("-----------------------HOF : Subtract-----------------------")
df_final, df_target = generate_raw_data(df_hof, df_diff_hof, df_same_hof, "Subtract", "HOF")
RawData = np.transpose(df_final.as_matrix())
RawTarget = df_target.as_matrix()
train_model(RawData, RawTarget, "HOF")

df_gsc = pd.read_csv("GSC/GSC-Features.csv")
df_diff_gsc = pd.read_csv("GSC/diffn_pairs.csv")
df_same_gsc = pd.read_csv("GSC/same_pairs.csv")

print("-----------------------GSC : Concat-----------------------")
df_final, df_target = generate_raw_data(df_gsc, df_diff_gsc, df_same_gsc, "Concat", "GSC")
RawData = np.transpose(df_final.as_matrix())
RawTarget = df_target.as_matrix()
train_model(RawData, RawTarget, "GSC")

print("-----------------------GSC : Subtract-----------------------")
df_final, df_target = generate_raw_data(df_gsc, df_diff_gsc, df_same_gsc, "Subtract", "GSC")
RawData = np.transpose(df_final.as_matrix())
RawTarget = df_target.as_matrix()
train_model(RawData, RawTarget, "GSC")

-----------------------HOF : Concat--------------------
BigSigma Generated..


/Users/atrayeenag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/Users/atrayeenag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


(10,)
----------Gradient Descent Solution--------------------
E_rms Training   = 0.49934
E_rms Validation = 0.4988
E_rms Testing    = 0.49423
